**(C) 2020 David Haunschild** ([d.g.haunschild@se18.qmul.ac.uk](mailto:d.g.haunschild@se18.qmul.ac.uk) - [web](https://quantumvis.pro/)), all rights reserved.

# DIY Covid-19 Dashboard 

This dashboard utilises Python with the pandas module to analyze COVID-19 data in the UK.

In [17]:
import os.path, time
from datetime import datetime
import ipywidgets as wdg
import ipywidgets as widgets
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
from IPython.display import Markdown, display, HTML
from uk_covid19 import Cov19API

In [110]:
%matplotlib inline
plt.rcParams['figure.dpi'] = 100

In [111]:
def printmd(string):
    """Prints in markdown format"""
    display(Markdown(string))

In [112]:
'''Load initial data from disk'''
with open("timeseries.json", "rt") as INFILE:
    data=json.load(INFILE)
    
with open("gender.json", "rt") as INFILE:
    data1=json.load(INFILE)

In [113]:
#Meta data check for latest update
def last_update(file_name):
    return "Last update: %s" % time.ctime(os.path.getmtime(f"{file_name}"))

In [114]:
'''API'''
def access_api():
    
    try:
        
        filters = [
        'areaType=overview'
      ]
        structure = {
        "date": "date",
        "cases": "newCasesByPublishDate",
        "hospital": "newAdmissions",
        "deaths": "cumDeaths28DaysByDeathDateRate"    
      }
        api = Cov19API(filters=filters, structure=structure)
        timeseries=api.get_json()

        with open('timeseries.json', "wt") as OUTF:
            data = json.dump(timeseries, OUTF)

        return timeseries
    
    except :
        print('Error connecting to API!', last_update('timeseries.json'), flush = True)

In [115]:
'''API 2_Male_Female_Cases'''
def access_api1():
    
    try:
        
        filters = [
        'areaType=nation'
        #'areaName=England'
      ]
        structure = {
            "males": "maleCases",
            "females": "femaleCases"    
      }
        api = Cov19API(filters=filters, structure=structure)
        gender=api.get_json()

        with open('gender.json', "wt") as OUTF:
            data = json.dump(gender, OUTF)

        return gender
    
    except :
        print('Error connecting to API!', last_update('timeseries.json'), flush = True)

In [116]:
'''Wrangle_DATA'''
def wrangle_data(data):
    datalist=data['data']#[0]
    dates=[dictionary['date'] for dictionary in datalist ]
    dates.sort()
    def parse_date(datestring):
        """ Convert a date string into a pandas datetime object """
        return pd.to_datetime(datestring, format="%Y-%m-%d")
    startdate=parse_date(dates[0])
    enddate=parse_date(dates[-1])
    index=pd.date_range(startdate, enddate, freq='D')
    timeseriesdf=pd.DataFrame(index=index, columns=['cases', 'hospital', 'deaths'])
    for entry in datalist: # each entry is a dictionary with date, cases, hospital and deaths
        date=parse_date(entry['date'])
        for column in ['cases', 'hospital', 'deaths']:
        
            if pd.isna(timeseriesdf.loc[date, column]): 
           
                value= float(entry[column]) if entry[column]!=None else 0.0
           
                timeseriesdf.loc[date, column]=value
            
    timeseriesdf.fillna(0.0, inplace=True)
    
    #pickle file
    timeseriesdf.to_pickle("timeseriesdf.pkl")
    timeseriesdf=pd.read_pickle("timeseriesdf.pkl")
            
    return timeseriesdf    

In [117]:
'''Wrangle_DATA2_Male_Female_Chart'''
def wrangle_data1(data):
    datadic=data['data'][0]
    males=datadic['males']
    females=datadic['females']
    ageranges=[x['age'] for x in males]
    
    def min_age(agerange):
        agerange=agerange.replace('+','') # remove the + from 90+
        start=agerange.split('_')[0]
        return int(start)
    
    ageranges.sort(key=min_age)
    age_df=pd.DataFrame(index=ageranges, columns=['males','females', 'total'])
    for entry in males: # each entry is a dictionary
        ageband=entry['age'] # our index position
        age_df.loc[ageband, 'males']=entry['value']
    
    for entry in females:
        ageband=entry['age']
        age_df.loc[ageband, 'females']=entry['value']
    
    age_df['total']=age_df['males']+age_df['females']
    
    #and create a pickle file
    age_df.to_pickle("genderdf.pkl")
    age_df=pd.read_pickle("genderdf.pkl")
            
    return age_df

In [118]:
'''Meta data check function for latest update'''
#Meta data check for latest update
def last_update(file_name):
    return "Last update: %s" % time.ctime(os.path.getmtime(f"{file_name}"))

In [132]:
'''API REFRESH BUTTON FUNCTION AND EXECUTE'''
#Cereating the output
output1 = wdg.Output()

@output1.capture(clear_output=True,wait=True)

def api_button_callback(button):
    
    try:
        
        #refeshing graph 1 ==> timeseries
        apidata = access_api()
        global df
        
        df = wrangle_data(apidata)

        refresh_graph()
        
        
        #refeshing graph 2 ==> demographics
        apidata1 = access_api1()
        
        df = wrangle_data1(apidata1)
        
        refresh_graph1()
        
        #refeshing graph 3 ==> Pie_Chart demographics
        
        refresh_graph2()
        
        
        #create a text box in dashboard
        button.icon = 'check'
        button.disabled = True
        button_style='success'
                  
        printmd('Success! '+ last_update('timeseries.json'))
                     
    
             
    except Exception:
        #Called when API fails to load!
        printmd('Error! ' + last_update('timeseries.json'))
        button.icon = 'warning'
        button_style='warning'
        button.description='Error API data!'
        button.disabled = False
    
    
    
apibutton=wdg.Button(
    description='Update Data', 
    disabled=False,
    button_style='Info', # 'success', 'info', 'warning', 'danger' or ''
    tooltip="download",
    icon='download'
)


apibutton.on_click(api_button_callback)

display(apibutton)
output1

Button(button_style='info', description='Update Data', icon='download', style=ButtonStyle(), tooltip='download…

Output()

## Graphs and Analysis

### Total UK Daily COVID-19 Statistics

The following graph shows the daily cumulative COVID-19 cases in the UK since the pandemic. As of November 2020, over 50,000 deaths were recorded, and more than one million cases of which over 200,000 were admitted to the hospital.

In [68]:
'''Graph Controls'''
interact = wdg.SelectMultiple(
    options = ["cases","hospital", "deaths"],
    value = ["cases","hospital", "deaths"],
    rows = 3,
    description = 'Chose Data:',
    disabled = False
)

log = wdg.RadioButtons(
    options = ['linear', 'logarithmic'],
    description = 'Type:',
    disabled = False
)

controls = wdg.HBox([interact, log])

In [11]:
'''Calling the File from above as 'df1'''
df1 = wrangle_data(data)

In [49]:
"""Plot timeseries (df1) + title"""
def timeseries(lines, scale):
    if scale=='linear':
        logscale=False
    else:
        logscale=True
    ncols=len(lines)
    if ncols>0:
        df1[list(lines)].plot(logy=logscale) #df1 from timeseries
        plt.xlabel('Month') 
        plt.ylabel('Number of Cases')
        plt.title("UK Daily Total COVID-19 Statistics")
        
    else:
        print("Select desired data.")
        print("(CTRL-Click to select more than one category)")

In [50]:
'''Throwing out the Graph 'timeseries' '''
plot_1 = wdg.interactive_output(timeseries, {'lines':interact, 'scale': log})

display(controls, plot_1)

Output()

Data: [Source](https://coronavirus.data.gov.uk/)

In [14]:
'''Refresh Graph Function 1'''
def refresh_graph():
    current = interact.value
    if current==interact.options[:]:
        other=[]
        interact.value=other
        interact.value=current
    else:
        other=interact.options[:]
        interact.value=other

In [48]:
#Second Graph

The folowing graph shows the COVID-19 cases based on dempgraphics in the UK since the pandemic.

In [136]:
'''Graph_2 Controls'''
interact1 = wdg.SelectMultiple(
    options=['males', 'females', 'total'], 
    value=['males', 'females'], 
    rows=3,
    description = 'Chose Data:',
    disabled = False
)

controls1 = wdg.HBox([interact1])

In [137]:
'''Calling the File from above as 'df2'''
df2 = wrangle_data1(data1)

In [138]:
"""Plot Male_Female_Chart (df2) + title"""
def demographics(lines):
    if lines=='linear':
        logscale=False
    else:
        logscale=True
    ncols=len(lines)
    if ncols>0:
        df2[list(lines)].plot(kind='barh', y=list(lines))    #plot(kind='bar', y=['males','females','total']) #df2 from Male_Female_Chart, as bar chart plotting
        plt.xlabel('Number of Cases') 
        plt.ylabel('Age')
        plt.title("UK COVID-19 Demographics")
    
    else:
        print("Select desired data.")
        print("(CTRL-Click to select more than one category)")

### UK  COVID-19 Demographics

The graph below shows the cases distributed among the age groups and males, females, and cumulatively. 

In [139]:
'''Throwing out the Graph 'Male_Female_Chart' '''
plot_2 = wdg.interactive_output(demographics, {'lines':interact1})

display(controls1, plot_2)

Output()

Data: [Source](https://coronavirus.data.gov.uk/)

In [66]:
'''Refresh Graph Function 2'''
def refresh_graph1():
    current = interact1.value
    if current==interact1.options[:]:
        other=[]
        interact1.value=other
        interact1.value=current
    else:
        other=interact1.options[:]
        interact1.value=other

The following chart shows a comparison between the total UK COVID-19 cases by Males and Females.

In [127]:
'''Graph_3 Controls'''
interact2 = wdg.SelectMultiple(
    options=['males', 'females'], 
    value=['males', 'females'], 
    rows=3,
    description = 'Chose Data:',
    disabled = True
)

In [128]:
"""Plot Male_Female_Pie_Chart (df2) + title"""
def pie(lines):
    if lines=='linear':
        logscale=False
    else:
        logscale=True
    ncols=len(lines)
    if ncols>0:
        demographics_sum = df2.sum()
        demographics_sum = demographics_sum.drop(index = ['total'])
        demographics_sum.plot.pie(y='females', figsize=(7, 7), legend=False, \
                   autopct='%1.1f%%', \
                   shadow=True, startangle=0)
        plt.xlabel('Total Cases in Percentage') 
        plt.ylabel('')
        plt.title("Total UK COVID-19 Cases by Demographics Based on Gender")
    
    else:
        print("Select desired data.")
        print("(CTRL-Click to select more than one category)")

Below the cumulative age groups, cases are compared by gender. As of late November 2020, more female COVID-19 cases than male cases in the UK are recorded.

In [129]:
'''Throwing out the Pie Chart 'Male_Female_Chart' '''
plot_3 = wdg.interactive_output(pie, {'lines':interact2})


display(plot_3)

Output()

Data: [Source](https://coronavirus.data.gov.uk/)

In [131]:
'''Refresh Pie Function 3'''
def refresh_graph2():
    current = interact2.value
    if current==interact2.options[:]:
        other=[]
        interact2.value=other
        interact2.value=current
    else:
        other=interact2.options[:]
        interact2.value=other

**(C) 2020 David Haunschild** ([d.g.haunschild@se18.qmul.ac.uk](mailto:d.g.haunschild@se18.qmul.ac.uk) - [web](https://quantumvis.pro/)), all rights reserved.


Created with help from materials by Fabrizio Smeraldi ([f.smeraldi@qmul.ac.uk](mailto:f.smeraldi@qmul.ac.uk) - [web](http://www.eecs.qmul.ac.uk/~fabri/))

Data source: *UK Government [data](https://coronavirus.data.gov.uk/) published by [Public Health England](https://www.gov.uk/government/organisations/public-health-england).*